In [ ]:
import dlib
import cv2
import numpy as np
import face_recognition
import pandas as pd

In [ ]:
train_data=pd.read_csv(r"train\annos.csv")
train_data.insert(train_data.shape[1],'EuDist',-1)
train_data.insert(train_data.shape[1],'SimDist',-1)


In [ ]:
train_data.at[ 1, "EuDist"]=1

In [ ]:
image = face_recognition.load_image_file(r"D:\1mylearningdata\1mphil\Competition\NCCCU2022\1my\train\data\452\a.jpg")
image = cv2.fastNlMeansDenoisingColored(image, None, 30, 30, 7, 21)

face_locations = face_recognition.face_locations(image)

img = image[face_locations[0][0]:face_locations[0][2], face_locations[0][3]:face_locations[0][1]]
face_encoding = face_recognition.face_encodings(image, face_locations)[0]

In [ ]:
a = np.asarray([ [1,2,3], [4,5,6], [7,8,9] ])
pd.DataFrame(a).to_csv('sample.csv')

In [ ]:
import dlib
import cv2
import numpy as np
import face_recognition
import pandas as pd
import os


def getFaceEncoding(src):
    image = face_recognition.load_image_file(src)
    image=cv2.resize(image,(image.shape[1]*2,image.shape[0]*2))
    image = cv2.medianBlur(image, 7)
    image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
    face_locations = face_recognition.face_locations(image)
    if face_locations == []:
        return np.array([])
    img = image[face_locations[0][0]:face_locations[0][2], face_locations[0][3]:face_locations[0][1]]
    face_encoding = face_recognition.face_encodings(image, face_locations)[0]
    return face_encoding


def getEuDist(img_encoding1, img_encoding2):
    img_encoding1 = np.array(img_encoding1)
    img_encoding2 = np.array(img_encoding2)
    diff = np.subtract(img_encoding1, img_encoding2)
    dist = np.sqrt(np.sum(np.square(diff)))
    return dist


def getSimDist(img_encoding1, img_encoding2):
    img_encoding1 = np.array(img_encoding1)
    img_encoding2 = np.array(img_encoding2)
    dist = np.linalg.norm(img_encoding1 - img_encoding2)
    sim = 1.0 / (1.0 + dist)
    return sim


id_imgGroup=[]
train_data=pd.read_csv(r"train\annos.csv")
train_data.insert(train_data.shape[1],'EuDist',np.nan)
train_data.insert(train_data.shape[1],'SimDist',np.nan)
train_path = "train\data"
file = os.walk(train_path)
for sub_path, sub_dir_list, sub_file_list in file:
    for sub_dir_name in sub_dir_list:
        sub_file = os.walk(os.path.join(sub_path, sub_dir_name))

        for sub2_path, sub2_dir_list, sub2_file_list in sub_file:
            img_encoding1 = getFaceEncoding(os.path.join(sub2_path, sub2_file_list[0]))
            img_encoding2 = getFaceEncoding(os.path.join(sub2_path, sub2_file_list[1]))
            if (img_encoding1.size == 0) or (img_encoding2.size == 0):
                continue
            else:
                print(sub_dir_name)
                eu_dist = getEuDist(img_encoding1, img_encoding2)
                sim_dist = getSimDist(img_encoding1, img_encoding2)
                train_data.at[int(sub_dir_name),"EuDist"]=eu_dist
                train_data.at[int(sub_dir_name),"SimDist"]=sim_dist
                # print(os.path.join(sub2_path, sub2_file_name))


In [ ]:
train_data.to_csv("train_data3.csv",index_label="id")